<a href="https://colab.research.google.com/github/bhavya2810/Bhavya_INFO5731/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **INFO5731 Assignment Three**


In this assignment, you are required to conduct information extraction, semantic analysis based on the dataset you collected from assignment two. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**


(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

In [42]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,353 kB]
Get:15 h

In [43]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

**Scrapping Data and Converting to Dataframe**

In [44]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome('chromedriver',options=options)
link = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
title_array = []
review_array = []
driver.get(link)
for num in range(4):
  driver.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(5)
  listOfTitle = driver.find_elements(By.CLASS_NAME, "title")
  listOfReviews = driver.find_elements(By.CLASS_NAME, "text")
for ele, sub_ele in zip(listOfTitle, listOfReviews):
      title_array.append((ele.text).replace('\n',''))
      review_array.append(sub_ele.text)
df = pd.DataFrame(list(zip(title_array, review_array)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 125


,Title,Review
0,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
1,Only certain people can relate,This is a movie that only those who have felt ...
2,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
3,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...
4,The Hype is real,Most of the time movies are anticipated like t...
...,...,...
120,Truly a masterpiece,"Truly a masterpiece, The Best Hollywood film o..."
121,Deserved Oscar!!,"The acting, cinematography, sound design and t..."
122,Don't waste your time or money,If you plan on watching this I would suggest s...
123,Horrible job of portraying the character,"If it wasn't for the title and make-up, you wo..."


In [45]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

# **Data Cleaning**

**Converting to Lower Case**

In [46]:
df['Lower Case'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))

**Removal of Punctuation**

In [47]:
df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')

**Removal of Special Characters**

In [48]:
import re
df['Removal of Special Characters'] = df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

**Removal of Stop words**

In [49]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Stopwords Removal'] = df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

**Spelling Correction**

In [50]:
from textblob import TextBlob
df['Spelling Correction'] = df['Stopwords Removal'].apply(lambda x: str(TextBlob(x).correct()))

**Tokenization**

In [51]:
import nltk
nltk.download('punkt')
df['Tokenization'] = df['Spelling Correction'].apply(lambda x: TextBlob(x).words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Stemming**

In [53]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['After Stemming'] = df['Tokenization'].apply(lambda x: " ".join([st.stem(word) for word in x]))

**Lemmatization**

In [54]:
from textblob import Word
import nltk
nltk.download('wordnet')

df['After Lemmatization'] = df['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.to_csv('/content/sample_data/reviwes.csv',index=False)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**1.1 - Trigrams**

In [55]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

words_sentence = []
for sentence in df['After Lemmatization']:
  words_sentence.append(word_tokenize(sentence))
aftercleaning = [x for x in words_sentence if x != []]
iterations = list(itertools.chain.from_iterable(aftercleaning))

In [56]:
tri_grams = nltk.trigrams(iterations)
frequency_dist = nltk.FreqDist(tri_grams)
frequency_dist

FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'hunt'): 1,
          ('grotesqu', 'hunt', 'cring'): 1,
          ('hunt', 'cring', 'hard'): 1,
          ('cring', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'wont'): 1,
          ('mesmer', 'wont', 'blink'): 1,
          ('wont', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'tragic'): 1,
          ('watch', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', 'moment

**1.2 - Probability of Bi grams**

In [57]:
from collections import Counter
bi_grams = nltk.bigrams(iterations)
frequency_dist = nltk.FreqDist(bi_grams)
bi_grams_dict = dict(frequency_dist)
for word in bi_grams_dict:
  print( str(word) + ':' + str(bi_grams_dict[word] / iterations.count(word[0])))

Streaming output truncated to the last 5000 lines.
('allie', 'classic'):1.0
('classic', 'base'):0.5
('base', 'review'):0.14285714285714285
('review', 'read'):0.03571428571428571
('read', 'end'):0.2
('end', 'watch'):0.11764705882352941
('watch', 'decent'):0.02040816326530612
('decent', 'film'):0.3333333333333333
('film', 'overal'):0.011494252873563218
('overal', 'phoenix'):0.14285714285714285
('phoenix', 'outstand'):0.015625
('outstand', 'definit'):0.3333333333333333
('definit', 'part'):0.08333333333333333
('part', 'feel'):0.07692307692307693
('like', 'you'):0.014925373134328358
('you', 'seen'):0.5
('seen', 'trailer'):0.04
('trailer', 'you'):0.14285714285714285
('movi', 'best'):0.008547008547008548
('best', 'way'):0.025
('way', 'describ'):0.03333333333333333
('describ', 'film'):0.5
('film', 'feel'):0.011494252873563218
('like', '2'):0.014925373134328358
('2', 'hour'):0.6363636363636364
('hour', 'version'):0.1
('version', 'trailer'):0.1111111111111111
('trailer', '2'):0.14285714285714285

**1.3 Noun Phrases and relative probabilities of each review in terms of other reviews**

In [58]:
whole_dataset = ''
index = []
i = 1
for line in df['After Lemmatization']:
  whole_dataset = whole_dataset + line
  value = 'Review-' + str(i)
  index.append(value)
  i += 1

In [59]:
from textblob import TextBlob
noun_phrases = []
frequency = []
for line in df['After Lemmatization']:
  blob = TextBlob(line)
  for nouns in blob.noun_phrases:
    noun_phrases.append(nouns)
for word in noun_phrases:
  noun_phrases_freq = []
  for line in df['After Lemmatization']:
    noun_phrases_freq.append(line.count(word) / whole_dataset.count(word))
  frequency.append(noun_phrases_freq)
noun_phrases_df = pd.DataFrame(frequency).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.index = index
noun_phrases_df

,everi movi,impact joaquin,scenographi brillianc grotesqu hunt,time mesmer wont blink eye,serious funni moment emot rollercoast sometim multipl emot,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,alon isol truli relat,sorri charact lot peopl,encourag violenc truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad truli,comic book willingli realist,direct cinematographi music act peopl surpris,violent necess messag societi reflect underappreciatedunrecognizedbulli peopl,someth way show class differ,corrupt rich talent rule,differ believ,multipl joke,bitter way film show,peopl comfort peopl consid wake,perfect film,joaquin phoenix,fearless stun emot depth physic imposs,referenc heath ledger oscarwin,dark knight,wide consid definit liveact portray joke,fact everyon,stun phoenix,imposs portray,potenti exce,dark knight clown princ crime,end fall,short way,short joke,time happi hope plea ignor complaint pernici violenc embarrass,...,bitter way film show,peopl comfort peopl consid wake,perfect film,act cinematographi,sound design script phenomenon film deserv oscar joke deserv,present venic film festiv event regard cinema form art film,movi film gene focus psychic,main charact slowli crumbl pressur societi thu joaquin phoenix wonder,sure nomin scar,joke differ ledger id,equal good main differ,ledger joke ration act insan phoenix insan root,30min otherwis complet bore,heath ledger joke,joke doesn potenti,movi show,mental issu,base comicbook charact,real joke weak confid pushov,genesi stage,real joke high toler pain,fight intellig,insan joaquin shoot guy subway cowardic tri,murray shoot scene way joaquin joke,shoe adam joke,differ inclin differ,comic authorsand,heath ledger portray,movi someth total differ,joaquin phoenix,joke surpris,pull hope,anoth chees condit,full fluff realist explan joke,adult son wouldn,nudg wake pleasantli surpris,phillip version play joaquin act superb,anguish charact,mind excel,act movi
Review-1,1.0,1.0,1.0,1.0,1.0,1.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.333333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.027027,1.0,1.0,0.181818,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Review-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.


# **Question 2: Undersand TF-IDF and Document representation**


(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program:

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

**2.1 - TF-IDF**

In [60]:
import math
def tf_idf_value(sentence,word):
  size = len(sentence.split(" "))
  tf_value = sentence.count(word)/size
  idf_value = 0
  if(tf_value!=0):
    idf_value = math.log(size)/sentence.count(word)
  else:
    return 0;
  return tf_value*idf_value

sentences = df["Spelling Correction"].values.tolist()
tokens_list = set([j for i in sentences for j in i.split(" ")])
tf_idf = pd.DataFrame(tokens_list,columns=["token"])
count = 0
for i in sentences:
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
  count+=1

tf_idf

,token,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124
0,,0.081918,0.093937,0.055855,0.08065,0.083231,0.058163,0.110774,0.035297,0.070931,0.0,0.0,0.061364,0.244136,0.188504,0.074911,0.244136,0.040702,0.071881,0.0,0.074911,0.058773,0.064221,0.069111,0.020991,0.0,0.0,0.0,0.25993,0.072861,0.0,0.039377,0.0,0.033765,0.093937,0.08065,0.116114,0.04126,0.0,0.108304,...,0.15497,0.173287,0.077095,0.149787,0.019224,0.055309,0.180537,0.125311,0.105955,0.054775,0.197304,0.113373,0.110774,0.027974,0.298627,0.048738,0.060693,0.047936,0.140502,0.067391,0.25993,0.128755,0.103716,0.064983,0.140502,0.149787,0.032879,0.062052,0.0,0.132419,0.088992,0.054252,0.0,0.149787,0.140502,0.055855,0.06348,0.116114,0.060693,0.067391
1,therapy,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,psychopath,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,confused,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.039377,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,took,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.040702,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.019224,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,face,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.

**2.2 - Ranking using Cosine Similarity**

In [61]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [62]:
query_sentence = 'The perfect portrait of a decline to insanity" & Joaquin Phoenix portrays this flawlessly in a performance of a lifetime. Choreographed impeccably by Todd Phillips & is nothing short of breathtaking. The cinematography & backdrop ideology of such an original film for such a complex character is only complimented by the superb soundtrack & acting. A hauntingly beautiful film & A Definate 10/10. Edit: Bring on part 2 plssssss!. We all want it!.'
X_list = word_tokenize(query_sentence)
sw = stopwords.words('english') 
X_set = {w for w in X_list if not w in sw}  

In [63]:
def compute_vector(X_set, Y_set):
  rvector = X_set.union(Y_set)
  l1 =[];l2 =[]
  for w in rvector: 
      if w in X_set: l1.append(1)
      else: l1.append(0) 
      if w in Y_set: l2.append(1) 
      else: l2.append(0)
  return rvector, l1, l2

In [64]:
def compute_cosine(rvector, l1, l2):
  c = 0
  for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

In [65]:
similarity_list = []
for sentence in df['Spelling Correction']:
  Y_list = word_tokenize(sentence)
  Y_set = {w for w in Y_list if not w in sw}
  rvector, l1, l2 = compute_vector(X_set, Y_set)
  try:
    similarity = compute_cosine(rvector, l1, l2)
  except ZeroDivisionError:
    similarity = 'None'
  similarity_list.append(similarity)
cosine_dataframe = pd.DataFrame(list(zip(df['Review'],similarity_list)), columns=['Reviews','Cosine Similarity'])
cosine_dataframe

,Reviews,Cosine Similarity
0,"Every once in a while a movie comes, that trul...",0.0434783
1,This is a movie that only those who have felt ...,0.0256664
2,"Truly a masterpiece, The Best Hollywood film o...",0.0761387
3,Joaquin Phoenix gives a tour de force performa...,0.0227508
4,Most of the time movies are anticipated like t...,0.0230266
...,...,...
120,"Truly a masterpiece, The Best Hollywood film o...",0.0761387
121,"The acting, cinematography, sound design and t...",0.0976458
122,If you plan on watching this I would suggest s...,0
123,"If it wasn't for the title and make-up, you wo...",0.0585875



# **Question 3: Create your own training and evaluation data for sentiment analysis**


(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification.

https://github.com/bhavya2810/Bhavya_INFO5731/blob/master/Reviews_Sentimental_Analysis.csv